In [55]:
import json
from pprint import pprint
import glob
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import lda

In [33]:
files = glob.glob('../data/AmazonReviews/TVs/*.json') # data from http://times.cs.uiuc.edu/~wang296/Data/
df_prods = pd.DataFrame()
df_revs = pd.DataFrame()
for i in files:
    with open(i) as data_file:
        f = json.load(data_file)
        try:
            temp_prods = pd.io.json.json_normalize(f['ProductInfo'])
            temp_revs = pd.io.json.json_normalize(f['Reviews'])
        except:
            pass
        
        df_prods = df_prods.append(temp_prods)
        df_revs = df_revs.append(temp_revs)


In [47]:
df_revs.dropna(axis=0, subset=['Content'], inplace=True)

In [41]:
cv = CountVectorizer(stop_words='english', max_df=.95, min_df=5)
X = cv.fit_transform(df_revs.Content)

In [ ]:
model = lda.LDA(n_topics=10, n_iter=1500, random_state=1)
model.fit(X)

<242638x33870 sparse matrix of type '<type 'numpy.int64'>'
	with 11445523 stored elements in Compressed Sparse Row format>